# MP305 Network Flow Models I

In [45]:
from IPython.display import display, Math, Latex

## Overview

This file contains a number of Python functions for finding the maximal flow through a network $G$ subject to minimal cost using the Ford Fulkerson Algorithm.

The network graph $G$ is stored in a set `G` of two element tuples `(i,j)` describing the directed arcs $(i,j)$ of $G$.

It is assumed that node number $1$ is the source and the greatest node `Nsink` is the sink.  
Thus `G={(1,2),(2,3),(1,3)}`  describes a network with 3 nodes where node 1 is the source and node 3 is the sink.

The capacity $c(i,j)$, flow $\phi(i,j)$ and cost $l(i,j)$ of the arc $(i,j)$ of $G$ are stored in `c[i][j]`, `phi[i][j]` and `l[i][j]`.
Here `c`, `phi` and `l` are Python lists.

## Python Functions
### The `Initialise(G)` function
Having defined the network $G$, initialise `c`, `phi` and `l` values to zero via the `Initialise` function before defining their values in any particular example.  The global variable `Nsink`,  the sink node of $G$, is also found by the `Initialise` function .

### The main `Iterate(G)`function
This implements the full algorithm to find the maximum flow with minimal cost.

## The `Iterate(G)` function is based on a number of other Python functions:

### `Flows(G)`
This checks for conservation of flow and prints out all of the current flows for G and the total cost of this flow.

###  `Links(G)`
This finds all arcs `(i,j1)`, ` (i,j2)`,  ... *out* of node `i` of `G`.  The nodes `j1,j2,..` are stored in a global list of sets `Out`.

###  `SourceSink(G)`
This finds all of the paths from source to sink in any network `G` and results in a global set `SinkPaths` of such paths.

###  `IncremNet(G)`
This finds the Incremental Network `Gp` associated with the current flow of the network `G`.

###  `Newflows(G)`
This updates the flows `phi` of `G` according to the best chain found through `Gp`. If the maximal flow is reached, then this is indicated and the maximum flow value is outputed. Otherwise, the output is: the change in flow (`eps`), the cost of the best chain, and the best chain.

###  `Iterate(G)`
Implements the full algorithm to find the maximum flow with minimal cost.
The output is as follows:

(1) The incremental network `Gp`.

(2) The paths through `Gp` from source to sink.

(3) The output of `Newflows(G)`.

(4) The output of `Flows(G)` giving the current flows and cost of `G`.


In [2]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")


In [3]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in Gin:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)


In [4]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [5]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [6]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

In [7]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost:
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", besteps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [8]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

# Example 1. The first example discussed in lectures.
## The capacity $c(i,j)$ and flow $\phi(i,j)$ are shown on each arc $(i,j)$

![Network](https://github.com/mcgettrick/mp305/blob/main/Lab1_1.jpg?raw=1)


### \* Find the incremental network and capacities at each iteration of the Ford Fulkerson algorithm.
### \* In the last iteration when the maximal flow is found,  identify which arcs are normal and which are inverted in the incremental network.
### \* Hence find the minimal capacity cut of this network flow model.

In [53]:
G={(1,2),(1,3),(2,3),(2,4),(2,5),(3,5),(4,6),(5,4),(5,6)}

In [54]:
Initialise (G)

All values of c,phi and l initialised to zero


In [55]:
Flows(G)

(2, 4)  has flow  0
(1, 2)  has flow  0
(5, 4)  has flow  0
(4, 6)  has flow  0
(2, 3)  has flow  0
(5, 6)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
(3, 5)  has flow  0
Total Cost is  0


## Define Capacities

In [56]:
c[1][2] = 4; c[1][3] = 5; c[2][3] = 2
c[2][4] = 1; c[2][5] = 4; c[3][5] = 3
c[4][6] = 2; c[5][4] = 2; c[5][6] = 6

## Define Flows

In [57]:
phi[1][2]=4; phi[1][3]=1; phi[2][3]=2; phi[2][4]=1; phi[2][5]=1
phi[3][5]=3; phi[4][6]=2; phi[5][4]=1; phi[5][6]=3

In [58]:
Flows(G)

(2, 4)  has flow  1
(1, 2)  has flow  4
(5, 4)  has flow  1
(4, 6)  has flow  2
(2, 3)  has flow  2
(5, 6)  has flow  3
(2, 5)  has flow  1
(1, 3)  has flow  1
(3, 5)  has flow  3
Total Cost is  0


## Use `Iterate` to run one iteration of the Ford Fulkerson Algorithm

In [59]:
Iterate(G)

Incremental Network: {(2, 1), (6, 5), (3, 1), (5, 4), (6, 4), (4, 2), (4, 5), (5, 6), (5, 3), (3, 2), (2, 5), (1, 3), (5, 2)}
Paths from source to sink:  {(1, 3, 2, 5, 6)}
A best path in Gp is  (1, 3, 2, 5, 6)  of minimum cost  0
The min capacity on this path is epsilon  2
The min cost is  0
(2, 4)  flow =  1
(1, 2)  flow =  4
(5, 4)  flow =  1
(4, 6)  flow =  2
(2, 3)  flow =  0
(5, 6)  flow =  5
(2, 5)  flow =  3
(1, 3)  flow =  3
(3, 5)  flow =  3


In [60]:
Iterate(G)

Incremental Network: {(2, 1), (6, 5), (3, 1), (5, 4), (6, 4), (4, 2), (2, 3), (4, 5), (5, 6), (5, 3), (2, 5), (1, 3), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 7  with minimal cost  0
(2, 4)  flow =  1
(1, 2)  flow =  4
(5, 4)  flow =  1
(4, 6)  flow =  2
(2, 3)  flow =  0
(5, 6)  flow =  5
(2, 5)  flow =  3
(1, 3)  flow =  3
(3, 5)  flow =  3


In [61]:
Iterate(G)

Incremental Network: {(2, 1), (6, 5), (3, 1), (5, 4), (6, 4), (4, 2), (2, 3), (4, 5), (5, 6), (5, 3), (2, 5), (1, 3), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 7  with minimal cost  0
(2, 4)  flow =  1
(1, 2)  flow =  4
(5, 4)  flow =  1
(4, 6)  flow =  2
(2, 3)  flow =  0
(5, 6)  flow =  5
(2, 5)  flow =  3
(1, 3)  flow =  3
(3, 5)  flow =  3


In [61]:
# Normal arcs: (5,4), (2,3), (5,6), (2,5), (1,3)
# Inverted arcs: (2,1), (6,5), (3,1), (6,4), (4,2), (4,5), (5,3), (5,2)

# Minimum cut : 7

## In the following two problems, first define the network and  its capacities following the template of problem 1 and then run the `Python`|code.




## Q.2 (*) . Find the maximal flow through the following network with the given capacities:
![Network](https://github.com/mcgettrick/mp305/blob/main/Lab1_2.jpg?raw=1)

In [62]:
G={(1,2),(1,3),(2,3),(3,2),(3,4),(4,2),(4,5),(2,5)}
Initialise (G)

All values of c,phi and l initialised to zero


##  Define the network `G` and the capacities `c[i][j]` for the given data and follow the same sequence of steps as for Q.1 with zero initial flow.
### \* Set the initial flow to zero at each arc and find the incremental network and capacities at each iteration of the Ford-Fulkerson algorithm.
### \* In the last iteration when the maximal flow is found,  identify which arcs are normal and which are inverted in the incremental network.
### \* Hence find the minimal capacity cut of this network flow model.

In [63]:
c[1][2] = 2; c[1][3] = 4; c[2][3] = 1
c[3][2] = 4; c[3][4] = 2; c[4][2] = 2
c[4][5] = 3; c[2][5] = 3;

In [64]:
phi[1][2] = 0; phi[1][3] = 0; phi[2][3] = 0
phi[3][2] = 0; phi[3][4] = 0; phi[4][2] = 0
phi[4][5] = 0; phi[2][5] = 0

In [65]:
Flows(G)

(1, 2)  has flow  0
(3, 4)  has flow  0
(4, 2)  has flow  0
(2, 3)  has flow  0
(4, 5)  has flow  0
(3, 2)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
Total Cost is  0


In [66]:
Iterate(G)

Incremental Network: {(2, 3), (4, 5), (1, 2), (3, 4), (3, 2), (2, 5), (1, 3), (4, 2)}
Paths from source to sink:  {(1, 3, 4, 2, 5), (1, 3, 4, 5), (1, 2, 5), (1, 2, 3, 4, 5), (1, 3, 2, 5)}
A best path in Gp is  (1, 3, 4, 2, 5)  of minimum cost  0
The min capacity on this path is epsilon  2
The min cost is  0
(1, 2)  flow =  0
(3, 4)  flow =  2
(4, 2)  flow =  2
(2, 3)  flow =  0
(4, 5)  flow =  0
(3, 2)  flow =  0
(2, 5)  flow =  2
(1, 3)  flow =  2


In [67]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (4, 3), (3, 1), (2, 3), (4, 5), (3, 2), (2, 5), (1, 3), (5, 2)}
Paths from source to sink:  {(1, 3, 2, 5), (1, 3, 2, 4, 5), (1, 2, 4, 5), (1, 2, 5)}
A best path in Gp is  (1, 3, 2, 5)  of minimum cost  0
The min capacity on this path is epsilon  1
The min cost is  0
(1, 2)  flow =  0
(3, 4)  flow =  2
(4, 2)  flow =  2
(2, 3)  flow =  0
(4, 5)  flow =  0
(3, 2)  flow =  1
(2, 5)  flow =  3
(1, 3)  flow =  3


In [68]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (4, 3), (3, 1), (2, 3), (4, 5), (3, 2), (1, 3), (5, 2)}
Paths from source to sink:  {(1, 3, 2, 4, 5), (1, 2, 4, 5)}
A best path in Gp is  (1, 3, 2, 4, 5)  of minimum cost  0
The min capacity on this path is epsilon  1
The min cost is  0
(1, 2)  flow =  0
(3, 4)  flow =  2
(4, 2)  flow =  1
(2, 3)  flow =  0
(4, 5)  flow =  1
(3, 2)  flow =  2
(2, 5)  flow =  3
(1, 3)  flow =  4


In [69]:
Iterate(G)

Incremental Network: {(2, 4), (1, 2), (4, 3), (3, 1), (5, 4), (4, 2), (2, 3), (4, 5), (3, 2), (5, 2)}
Paths from source to sink:  {(1, 2, 4, 5)}
A best path in Gp is  (1, 2, 4, 5)  of minimum cost  0
The min capacity on this path is epsilon  1
The min cost is  0
(1, 2)  flow =  1
(3, 4)  flow =  2
(4, 2)  flow =  0
(2, 3)  flow =  0
(4, 5)  flow =  2
(3, 2)  flow =  2
(2, 5)  flow =  3
(1, 3)  flow =  4


In [70]:
Iterate(G)

Incremental Network: {(1, 2), (2, 1), (4, 3), (3, 1), (5, 4), (4, 2), (2, 3), (4, 5), (3, 2), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 5  with minimal cost  0
(1, 2)  flow =  1
(3, 4)  flow =  2
(4, 2)  flow =  0
(2, 3)  flow =  0
(4, 5)  flow =  2
(3, 2)  flow =  2
(2, 5)  flow =  3
(1, 3)  flow =  4


In [ ]:
# Normal arcs: (1,2), (4,5), (4,2), (2,3), (3,2)
# Inverted arcs: (2,1), (4,3), (3,1), (5,4), (5,2)

#Minimum cut = 5

## Q.3 (*) A road network is shown below with the capacity on each road indicated. Notice that many roads are two way.
![Network](https://github.com/mcgettrick/mp305/blob/main/Lab1_3.jpg?raw=1)
### \* Find the maximal flow through the network from `A` to `B`.
### \* Compare this to maximal flow from `B`  to `A` .

In [20]:
G={(1,2),(2,1),(2,5),
   (1,3),(3,1),
   (4,6),(6,4),(4,2),
   (3,4),(4,3),(3,7),
   (7,8),(8,7),(7,6),
   (5,8),(8,5),(5,6)
   }
Initialise(G)

All values of c,phi and l initialised to zero


In [21]:
c[1][2] = 5; c[2][1] = 5; c[1][3] = 4
c[3][1] = 4; c[2][5] = 3; c[4][6] = 5
c[6][4] = 5; c[4][2] = 4; c[3][4] = 5
c[4][3] = 5; c[3][7] = 3; c[7][8] = 4
c[8][7] = 4; c[7][6] = 4; c[5][8] = 6
c[8][5] = 6; c[5][6] = 2;

In [73]:
phi[0][1] = 0; phi[0][2] = 0; phi[2][0] = 0
phi[1][0] = 0; phi[1][5] = 0; phi[2][3] = 0
phi[3][2] = 0; phi[3][4] = 0; phi[4][3] = 0
phi[2][6] = 0; phi[6][4] = 0; phi[5][4] = 0
phi[7][5] = 0; phi[5][7] = 0; phi[7][6] = 0
phi[3][1] = 0; phi[6][7] = 0;

In [22]:
Flows(G)

(1, 2)  has flow  0
(2, 1)  has flow  0
(3, 4)  has flow  0
(4, 3)  has flow  0
(3, 1)  has flow  0
(3, 7)  has flow  0
(4, 6)  has flow  0
(6, 4)  has flow  0
(7, 6)  has flow  0
(2, 5)  has flow  0
(1, 3)  has flow  0
(5, 8)  has flow  0
(8, 7)  has flow  0
(4, 2)  has flow  0
(5, 6)  has flow  0
(8, 5)  has flow  0
(7, 8)  has flow  0
Total Cost is  0


In [23]:
Iterate(G)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (3, 7), (4, 6), (6, 4), (7, 6), (2, 5), (1, 3), (5, 8), (8, 7), (4, 2), (5, 6), (8, 5), (7, 8)}
Paths from source to sink:  {(1, 3, 7, 8), (1, 2, 5, 8), (1, 2, 5, 6, 4, 3, 7, 8), (1, 3, 7, 6, 4, 2, 5, 8), (1, 3, 4, 2, 5, 8)}
A best path in Gp is  (1, 3, 7, 8)  of minimum cost  0
The min capacity on this path is epsilon  3
The min cost is  0
(1, 2)  flow =  0
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  0
(1, 3)  flow =  3
(5, 8)  flow =  0
(8, 7)  flow =  0
(4, 2)  flow =  0
(5, 6)  flow =  0
(8, 5)  flow =  0
(7, 8)  flow =  3


In [24]:
Iterate(G)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (4, 6), (6, 4), (7, 3), (7, 6), (2, 5), (1, 3), (5, 8), (8, 7), (4, 2), (5, 6), (8, 5), (7, 8)}
Paths from source to sink:  {(1, 2, 5, 8), (1, 3, 4, 2, 5, 8)}
A best path in Gp is  (1, 2, 5, 8)  of minimum cost  0
The min capacity on this path is epsilon  3
The min cost is  0
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  3
(1, 3)  flow =  3
(5, 8)  flow =  3
(8, 7)  flow =  0
(4, 2)  flow =  0
(5, 6)  flow =  0
(8, 5)  flow =  0
(7, 8)  flow =  3


In [25]:
Iterate(G)

Incremental Network: {(1, 2), (2, 1), (3, 4), (4, 3), (3, 1), (4, 6), (6, 4), (7, 3), (7, 6), (1, 3), (5, 2), (5, 8), (8, 7), (4, 2), (5, 6), (8, 5), (7, 8)}
Paths from source to sink:  set()
Maximal flow found: 6  with minimal cost  0
(1, 2)  flow =  3
(2, 1)  flow =  0
(3, 4)  flow =  0
(4, 3)  flow =  0
(3, 1)  flow =  0
(3, 7)  flow =  3
(4, 6)  flow =  0
(6, 4)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  3
(1, 3)  flow =  3
(5, 8)  flow =  3
(8, 7)  flow =  0
(4, 2)  flow =  0
(5, 6)  flow =  0
(8, 5)  flow =  0
(7, 8)  flow =  3


In [32]:
#Reverse flow from B to A
Grev={(1,2),(2,1),(1,4),(4,1),(2,5),(5,2),(4,5),(5,6),(6,5),(6,7),(7,6),(6,3),(7,8),(8,7),(8,3),(3,8),(7,4)}
Initialise(Grev)
Flows(Grev)

All values of c,phi and l initialised to zero
(1, 2)  has flow  0
(2, 1)  has flow  0
(6, 7)  has flow  0
(8, 3)  has flow  0
(7, 6)  has flow  0
(2, 5)  has flow  0
(4, 1)  has flow  0
(5, 2)  has flow  0
(7, 4)  has flow  0
(3, 8)  has flow  0
(6, 5)  has flow  0
(8, 7)  has flow  0
(1, 4)  has flow  0
(4, 5)  has flow  0
(5, 6)  has flow  0
(6, 3)  has flow  0
(7, 8)  has flow  0
Total Cost is  0


In [33]:
c[1][2] = 6; c[2][1] = 6; c[1][4] = 4
c[4][1] = 4; c[4][5] = 4; c[2][5] = 2
c[5][2] = 2; c[5][6] = 5; c[6][5] = 5
c[6][7] = 5; c[7][6] = 5; c[6][3] = 4
c[7][8] = 4; c[8][7] = 4; c[8][3] = 5
c[3][8] = 5; c[7][4] = 3;

In [34]:
Iterate(Grev)

Incremental Network: {(1, 2), (2, 1), (6, 7), (8, 3), (7, 6), (2, 5), (4, 1), (5, 2), (7, 4), (3, 8), (6, 5), (8, 7), (1, 4), (4, 5), (5, 6), (6, 3), (7, 8)}
Paths from source to sink:  {(1, 4, 5, 6, 7, 8), (1, 2, 5, 6, 7, 8), (1, 4, 5, 6, 3, 8), (1, 2, 5, 6, 3, 8)}
A best path in Gp is  (1, 4, 5, 6, 7, 8)  of minimum cost  0
The min capacity on this path is epsilon  4
The min cost is  0
(1, 2)  flow =  0
(2, 1)  flow =  0
(6, 7)  flow =  4
(8, 3)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  0
(4, 1)  flow =  0
(5, 2)  flow =  0
(7, 4)  flow =  0
(3, 8)  flow =  0
(6, 5)  flow =  0
(8, 7)  flow =  0
(1, 4)  flow =  4
(4, 5)  flow =  4
(5, 6)  flow =  4
(6, 3)  flow =  0
(7, 8)  flow =  4


In [35]:
Iterate(Grev)

Incremental Network: {(7, 4), (3, 8), (1, 2), (2, 1), (6, 5), (8, 7), (5, 4), (6, 7), (8, 3), (7, 6), (6, 3), (5, 6), (2, 5), (4, 1), (5, 2)}
Paths from source to sink:  {(1, 2, 5, 6, 3, 8)}
A best path in Gp is  (1, 2, 5, 6, 3, 8)  of minimum cost  0
The min capacity on this path is epsilon  1
The min cost is  0
(1, 2)  flow =  1
(2, 1)  flow =  0
(6, 7)  flow =  4
(8, 3)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  1
(4, 1)  flow =  0
(5, 2)  flow =  0
(7, 4)  flow =  0
(3, 8)  flow =  1
(6, 5)  flow =  0
(8, 7)  flow =  0
(1, 4)  flow =  4
(4, 5)  flow =  4
(5, 6)  flow =  5
(6, 3)  flow =  1
(7, 8)  flow =  4


In [36]:
SourceSink(Grev)

Paths from source to sink:  {(1, 4, 5, 6, 7, 8), (1, 2, 5, 6, 7, 8), (1, 4, 5, 6, 3, 8), (1, 2, 5, 6, 3, 8)}


In [37]:
Iterate(Grev)

Incremental Network: {(7, 4), (3, 8), (1, 2), (2, 1), (6, 5), (8, 7), (5, 4), (6, 7), (8, 3), (7, 6), (6, 3), (3, 6), (2, 5), (4, 1), (5, 2)}
Paths from source to sink:  set()
Maximal flow found: 5  with minimal cost  0
(1, 2)  flow =  1
(2, 1)  flow =  0
(6, 7)  flow =  4
(8, 3)  flow =  0
(7, 6)  flow =  0
(2, 5)  flow =  1
(4, 1)  flow =  0
(5, 2)  flow =  0
(7, 4)  flow =  0
(3, 8)  flow =  1
(6, 5)  flow =  0
(8, 7)  flow =  0
(1, 4)  flow =  4
(4, 5)  flow =  4
(5, 6)  flow =  5
(6, 3)  flow =  1
(7, 8)  flow =  4


In [ ]:
# A->B Maximal flow : 6
# B->A Maximal flow : 5